<a href="https://colab.research.google.com/github/Yunusfahreza/yresolusi/blob/main/Tf_Idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy==3.*
!python -m spacy download en_core_web_sm
!python -m spacy info

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.

============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        



In [ ]:
import spacy

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Fetching Datasets

corpus = fetch_20newsgroups(categories=['sci.space'],
                            remove=('headers', 'footers', 'quotes'))

In [ ]:
print(type(corpus))

<class 'sklearn.utils._bunch.Bunch'>


In [ ]:
# Number of posts in our dataset.
len(corpus.data)

593

In [ ]:
# View first two posts.
corpus.data[:2]

["\nAny lunar satellite needs fuel to do regular orbit corrections, and when\nits fuel runs out it will crash within months.  The orbits of the Apollo\nmotherships changed noticeably during lunar missions lasting only a few\ndays.  It is *possible* that there are stable orbits here and there --\nthe Moon's gravitational field is poorly mapped -- but we know of none.\n\nPerturbations from Sun and Earth are relatively minor issues at low\naltitudes.  The big problem is that the Moon's own gravitational field\nis quite lumpy due to the irregular distribution of mass within the Moon.",
 '\nGlad to see Griffin is spending his time on engineering rather than on\nritual purification of the language.  Pity he got stuck with the turkey\nrather than one of the sensible options.']

In [ ]:
# Creating Tf-Idf Features

# Like before, if we want to use spaCy's tokenizer, we need
# to create a callback. Remember to upgrade spaCy if you need
# to (refer to beginnning of file for commentary and instructions).
nlp = spacy.load('en_core_web_sm')

# We don't need named-entity recognition nor dependency parsing for
# this so these components are disabled. This will speed up the
# pipeline. We do need part-of-speech tagging however.
unwanted_pipes = ["ner", "parser"]

# For this exercise, we'll remove punctuation and spaces (which
# includes newlines), filter for tokens consisting of alphabetic
# characters, and return the lemma (which require POS tagging).
def spacy_tokenizer(doc):
  with nlp.disable_pipes(*unwanted_pipes):
    return [t.lemma_ for t in nlp(doc) if \
            not t.is_punct and \
            not t.is_space and \
            t.is_alpha]

In [ ]:
%%time
# Use the default settings of TfidfVectorizer.
vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
features = vectorizer.fit_transform(corpus.data)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 19.2 s, sys: 442 ms, total: 19.6 s
Wall time: 24.4 s


In [ ]:
# The number of unique tokens.
print(len(vectorizer.get_feature_names_out()))

9440


In [ ]:
# The dimensions of our feature matrix. X rows (documents) by Y columns (tokens).
print(features.shape)

(593, 9440)


In [ ]:
# What the encoding of the first document looks like in sparse format.
print(features[0])

  (0, 5064)	0.10452754121963853
  (0, 2351)	0.12747025764625855
  (0, 4340)	0.15331700873692364
  (0, 2459)	0.10862435105627101
  (0, 4916)	0.17102715751031994
  (0, 6702)	0.09940033595823265
  (0, 5982)	0.10183554382071024
  (0, 6514)	0.08455482269873241
  (0, 896)	0.0892999596249832
  (0, 316)	0.1109487112663238
  (0, 4896)	0.08247641364333849
  (0, 628)	0.051044670776703174
  (0, 4368)	0.10270174012167517
  (0, 5274)	0.13259746290766442
  (0, 6908)	0.12524708704889775
  (0, 2494)	0.07376562213268434
  (0, 8105)	0.09513204666042695
  (0, 3287)	0.051874685324429695
  (0, 6181)	0.1390186329543497
  (0, 5652)	0.11219531673533985
  (0, 4589)	0.06321728493925476
  (0, 9158)	0.06158004812009137
  (0, 1141)	0.048918909156680825
  (0, 5023)	0.12320196834845284
  (0, 6354)	0.15331700873692364
  :	:
  (0, 1344)	0.09036471134545682
  (0, 5403)	0.17102715751031994
  (0, 451)	0.10452754121963853
  (0, 5790)	0.0991335109087398
  (0, 8368)	0.20402991671500817
  (0, 5377)	0.10099775257415368
  (0, 9

In [ ]:
# Querying The Data

# Transform the query into a TF-IDF vector.
query = ["lunar orbit"]
query_tfidf = vectorizer.transform(query)

In [ ]:
# Calculate the cosine similarities between the query and each document.
# We're calling flatten() here becaue cosine_similarity returns a list
# of lists and we just want a single list.
cosine_similarities = cosine_similarity(features, query_tfidf).flatten()

In [ ]:
import numpy as np

# numpy's argsort() method returns a list of *indices* that
# would sort an array:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
#
# The sort is ascending, but we want the largest k cosine_similarites
# at the bottom of the sort. So we negate k, and get the last k
# entries of the indices list in reverse order. There are faster
# ways to do this using things like argpartition but this is
# more succinct.
def top_k(arr, k):
  kth_largest = (k + 1) * -1
  return np.argsort(arr)[:kth_largest:-1]

In [ ]:
# So for our query above, these are the top five documents.
top_related_indices = top_k(cosine_similarities, 5)
print(top_related_indices)

[249 108   0 312 509]


In [ ]:
# Let's take a look at their respective cosine similarities.
print(cosine_similarities[top_related_indices])

[0.47855355 0.4292246  0.2736328  0.19486489 0.19125175]


In [ ]:
# Top match.
print(corpus.data[top_related_indices[0]])


Actually, Hiten wasn't originally intended to go into lunar orbit at all,
so it indeed didn't have much fuel on hand.  The lunar-orbit mission was
an afterthought, after Hagoromo (a tiny subsatellite deployed by Hiten
during a lunar flyby) had a transmitter failure and its proper insertion
into lunar orbit couldn't be positively confirmed.

It should be noted that the technique does have disadvantages.  It takes
a long time, and you end up with a relatively inconvenient lunar orbit.
If you want something useful like a low circular polar orbit, you do have
to plan to expend a certain amount of fuel, although it is reduced from
what you'd need for the brute-force approach.


In [ ]:
# Second-best match.
print(corpus.data[top_related_indices[1]])


Their Hiten engineering-test mission spent a while in a highly eccentric
Earth orbit doing lunar flybys, and then was inserted into lunar orbit
using some very tricky gravity-assist-like maneuvering.  This meant that
it would crash on the Moon eventually, since there is no such thing as
a stable lunar orbit (as far as anyone knows), and I believe I recall
hearing recently that it was about to happen.


In [ ]:
# Try a different query
query = ["satellite"]
query_tfidf = vectorizer.transform(query)

cosine_similarities = cosine_similarity(features, query_tfidf).flatten()
top_related_indices = top_k(cosine_similarities, 5)

print(top_related_indices)
print(cosine_similarities[top_related_indices])

[378 138 248 539  61]
[0.39068985 0.34073761 0.29838056 0.26242297 0.25695438]


In [ ]:
print(corpus.data[top_related_indices[0]])



As an Amateur Radio operator (VHF 2metres) I like to keep up with what is 
going up (and for that matter what is coming down too).
 
In about 30 days I have learned ALOT about satellites current, future and 
past all the way back to Vanguard series and up to Astro D observatory 
(space).  I borrowed a book from the library called Weater Satellites (I 
think, it has a photo of the earth with a TIROS type satellite on it.)
 
I would like to build a model or have a large color poster of one of the 
TIROS satellites I think there are places in the USA that sell them.
ITOS is my favorite looking satellite, followed by AmSat-OSCAR 13 
(AO-13).
 
TTYL
73
Jim


In [ ]:
#
# EXERCISE: fetch multiple topics from the 20 newsgroups
# dataset and query them using the approach we followed.
# A list of topics can be found here:
# https://scikit-learn.org/stable/datasets/real_world.html#the-20-newsgroups-text-dataset
#
# If you're feeling ambitious, incorporate n-grams or
# look at how you can measure precision and recall.
#

# Fetch the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='train')

# Select multiple topics of interest
topics = ['sci.space', 'rec.sport.baseball', 'comp.graphics']

# Filter documents related to the selected topics
selected_documents = [doc for doc, target in zip(newsgroups.data, newsgroups.target) if newsgroups.target_names[target] in topics]

# Preprocess the text data and compute TF-IDF representation
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000, ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(selected_documents)

# Sample query document
query_document = "I'm interested in baseball statistics and graphics design."

# Compute TF-IDF representation of the query document
query_tfidf = vectorizer.transform([query_document])

# Query the dataset using cosine similarity
similarities = cosine_similarity(tfidf_matrix, query_tfidf)

# Sort the similarities and retrieve top documents
top_indices = similarities.argsort(axis=0)[::-1][:5].flatten()

# Print the top documents and their similarities
for idx in top_indices:
    print(f"Similarity: {similarities[idx][0]}")
    print(selected_documents[idx])
    print("="*80)


Similarity: 0.28857607178332506
From: jaskew@spam.maths.adelaide.edu.au (Joseph Askew)
Subject: Re: the call to space (was Re: Clueless Szaboisms )
Organization: Statistics, Pure & Applied Mathematics, University of Adelaide
Lines: 34

In article <1pfiuh$64e@access.digex.com> prb@access.digex.com (Pat) writes:

>If the japanese are really going for Nukes, why not go with better
>technology then we have.  AS opposed to BWR/PWRs  have they really
>considered some of the 3rd generation Inherently safe designs.

The Japanese are still on the learning curve as far as nuclear power goes.
This means that unlike the Germans (who do great things all by themselves)
the Japanese tie up with foreign companies. The major one is Mitsubishi
(who else) who have a sharing agreement with GE I think. No chance of a
new design.

>Sodium has lots of chemical problems but it really solves design
>difficulties.  Or the inherently safe types.

Sodium has *lots* of chemical problems. Like it eats stainless ste